In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-tamil/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

12954

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [12]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [13]:
remove_duplicate('விமான போக்குவரத்து தொரையில் பாதுகாப்பு அம்சங்கள் உலகத்தரத்தில் உள்ள போதும் அவற்றை மேலும் மேம்படுத்த வேண்டியது அவசியம் இன்று குறிப்பிட்டார்.')

'விமான போக்குவரத்து தொரையில் பாதுகாப்பு அம்சங்கள் உலகத்தரத்தில் உள்ள போதும் அவற்றை மேலும் மேம்படுத்த வேண்டியது அவசியம் இன்று குறிப்பிட்டார்.'

In [14]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_ta'] > minimum_score:
                a = np.array(data[i]['predict_ta'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        if len(t_) > 100:
                            t_ = remove_duplicate(t_)
                            
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ta|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [15]:
results = loop((files[:10], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 10/10 [00:13<00:00,  1.31s/it]


In [16]:
len(results)

415

In [20]:
results[-4]['new_text']

'<|startoftranscript|><|ta|><|transcribe|><|0.00|>ஷாங்கை மாஸ்டர்ஸ் தெனிஸ் போட்டிகின் இருதிகாட்டத்தில் செர்பியாவின் நோவக் ஜோகோவிச் 6-3, 6-4 என்ற செட்கனக்கில் கிரோஷியாவின் போர்ணா கோரிக்கை வெண்டார்.<|8.70|><|endoftext|>'

In [21]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
tokenizer.tokenize(results[-1]['new_text'])

['<|startoftranscript|>',
 '<|ta|>',
 '<|transcribe|>',
 '<|0.00|>',
 'Ġà®ĩ',
 'à®¨',
 'à¯į',
 'à®¤',
 'Ġà®ĩ',
 'à®£',
 'à¯Ī',
 'Ġà®',
 'ı',
 'à®°',
 'à®¿',
 'à®¯',
 'à®²',
 'à¯į',
 'Ġà®ª',
 'à¯Ĩ',
 'à®ķ',
 'à®°',
 'à¯į',
 'Ġà®',
 '°',
 'à®¾',
 'à®ª',
 'à®°',
 'à¯į',
 'à®Ł',
 'à¯į',
 'à®Ł',
 'à¯ĭ',
 'Ġà®ķ',
 'à¯ģ',
 'à®¯',
 'à®¿',
 'à®°',
 'à¯ĭ',
 'à®',
 '¸',
 'à¯į',
 'Ġà®ĩ',
 'à®£',
 'à¯Ī',
 'à®¯',
 'à¯Ī',
 'Ġ4',
 '-',
 '6',
 ',',
 'Ġ6',
 '-',
 '3',
 ',',
 'Ġ10',
 '-',
 '5',
 'Ġà®İà®©',
 'à¯į',
 'à®±',
 'à¯ģ',
 'Ġà®ļ',
 'à®¿',
 'à®±',
 'à¯į',
 'à®ķ',
 'à®©',
 'à®ķ',
 'à¯į',
 'à®ķ',
 'à®¿',
 'à®²',
 'à¯į',
 'Ġà®µ',
 'à¯Ģ',
 'à®Ł',
 'à¯į',
 'à®Ł',
 'à®¿',
 'à®¯',
 'à®¤',
 'à¯ģ.',
 '<|5.64|>',
 '<|endoftext|>']

In [23]:
tokenizer.decode(tokenizer(results[-1]['new_text'])['input_ids'], decode_with_timestamps = True)

'<|startoftranscript|><|notimestamps|><|startoftranscript|><|ta|><|transcribe|><|0.00|> இந்த இணை ஏரியல் பெகர் ராபர்ட்டோ குயிரோஸ் இணையை 4-6, 6-3, 10-5 என்று சிற்கனக்கில் வீட்டியது.<|5.64|><|endoftext|><|endoftext|>'

In [1]:
results = mp.multiprocessing(files, loop, cores = 100)

In [25]:
len(results)

532262

In [26]:
results[0]

{'new_text': '<|startoftranscript|><|ta|><|transcribe|><|0.00|>விமான போக்குவரத்து தொரையில் பாதுகாப்பு அம்சங்கள் உலகத்தரத்தில் உள்ள போதும் அவற்றை மேலும் மேம்படுத்த வேண்டியது அவசியம் இன்று குறிப்பிட்டார்.<|8.20|><|endoftext|>',
 'audio_filename': '/home/ubuntu/newsonair_v5/tamil/NSD-Tamil-Tamil-0715-0725-201810118533/sent_13.wav'}

In [27]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [28]:
with open('prepared-tamil.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 532262/532262 [00:02<00:00, 209442.82it/s]


In [29]:
!ls -lh prepared-tamil.jsonl

-rw-r--r-- 1 ubuntu ubuntu 411M Apr 27 10:16 prepared-tamil.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj='prepared-tamil.jsonl',
    path_in_repo='prepared-tamil.jsonl',
    repo_id='mesolitica/pseudolabel-tamil-large-v3-timestamp',
    repo_type='dataset',
)

prepared-tamil.jsonl:   0%|          | 0.00/430M [00:00<?, ?B/s]